In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from catboost import CatBoostClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,roc_auc_score
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, cross_validate
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from yellowbrick.cluster import KElbowVisualizer

In [2]:
import warnings
warnings.simplefilter(action="ignore")
warnings.filterwarnings("ignore", category=UserWarning, module="lightgbm")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 170)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [3]:
df = pd.read_csv('/Users/simayozkan/Downloads/creditcard.csv')

In [4]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.000,-1.360,-0.073,2.536,1.378,-0.338,0.462,0.240,0.099,0.364,0.091,-0.552,-0.618,-0.991,-0.311,1.468,-0.470,0.208,0.026,0.404,0.251,-0.018,0.278,-0.110,0.067,0.129,-0.189,0.134,-0.021,149.620,0
1,0.000,1.192,0.266,0.166,0.448,0.060,-0.082,-0.079,0.085,-0.255,-0.167,1.613,1.065,0.489,-0.144,0.636,0.464,-0.115,-0.183,-0.146,-0.069,-0.226,-0.639,0.101,-0.340,0.167,0.126,-0.009,0.015,2.690,0
2,1.000,-1.358,-1.340,1.773,0.380,-0.503,1.800,0.791,0.248,-1.515,0.208,0.625,0.066,0.717,-0.166,2.346,-2.890,1.110,-0.121,-2.262,0.525,0.248,0.772,0.909,-0.689,-0.328,-0.139,-0.055,-0.060,378.660,0
3,1.000,-0.966,-0.185,1.793,-0.863,-0.010,1.247,0.238,0.377,-1.387,-0.055,-0.226,0.178,0.508,-0.288,-0.631,-1.060,-0.684,1.966,-1.233,-0.208,-0.108,0.005,-0.190,-1.176,0.647,-0.222,0.063,0.061,123.500,0
4,2.000,-1.158,0.878,1.549,0.403,-0.407,0.096,0.593,-0.271,0.818,0.753,-0.823,0.538,1.346,-1.120,0.175,-0.451,-0.237,-0.038,0.803,0.409,-0.009,0.798,-0.137,0.141,-0.206,0.502,0.219,0.215,69.990,0


In [5]:
def check_df(dataframe, head=5):
    print("##################### Shape #####################")
    print(dataframe.shape)
    print("##################### Types #####################")
    print(dataframe.dtypes)
    print("##################### Head #####################")
    print(dataframe.head(head))
    print("##################### Tail #####################")
    print(dataframe.tail(head))
    print("##################### NA #####################")
    print(dataframe.isnull().sum())
 

check_df(df)

##################### Shape #####################
(284807, 31)
##################### Types #####################
Time      float64
V1        float64
V2        float64
V3        float64
V4        float64
V5        float64
V6        float64
V7        float64
V8        float64
V9        float64
V10       float64
V11       float64
V12       float64
V13       float64
V14       float64
V15       float64
V16       float64
V17       float64
V18       float64
V19       float64
V20       float64
V21       float64
V22       float64
V23       float64
V24       float64
V25       float64
V26       float64
V27       float64
V28       float64
Amount    float64
Class       int64
dtype: object
##################### Head #####################
   Time     V1     V2    V3     V4     V5     V6     V7     V8     V9    V10    V11    V12    V13    V14    V15    V16    V17    V18    V19    V20    V21    V22    V23  \
0 0.000 -1.360 -0.073 2.536  1.378 -0.338  0.462  0.240  0.099  0.364  0.091 -0.552 -0.618 -0.9

In [6]:
#Fraud = 1, Non-Fraud = 0
df["Class"].value_counts()

Class
0    284315
1       492
Name: count, dtype: int64

In [7]:
#n_f = Non_Fraud data. f = Fraud
n_f = df[df.Class == 0]
f = df[df.Class == 1]

In [8]:
print(n_f.shape)

(284315, 31)


In [9]:
print(f.shape)

(492, 31)


In [10]:
n_f.Amount.describe()

count   284315.000
mean        88.291
std        250.105
min          0.000
25%          5.650
50%         22.000
75%         77.050
max      25691.160
Name: Amount, dtype: float64

In [11]:
f.Amount.describe()

count    492.000
mean     122.211
std      256.683
min        0.000
25%        1.000
50%        9.250
75%      105.890
max     2125.870
Name: Amount, dtype: float64

In [12]:
df.groupby("Class").mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,94838.202,0.008,-0.006,0.012,-0.008,0.005,0.002,0.010,-0.001,0.004,0.010,-0.007,0.011,0.000,0.012,0.000,0.007,0.012,0.004,-0.001,-0.001,-0.001,-0.000,0.000,0.000,-0.000,-0.000,-0.000,-0.000,88.291
1,80746.807,-4.772,3.624,-7.033,4.542,-3.151,-1.398,-5.569,0.571,-2.581,-5.677,3.800,-6.259,-0.109,-6.972,-0.093,-4.140,-6.666,-2.246,0.681,0.372,0.714,0.014,-0.040,-0.105,0.041,0.052,0.171,0.076,122.211


In [13]:
n_f_sample = n_f.sample(n = 492)

In [14]:
new_df = pd.concat([n_f_sample, f], axis = 0)

In [15]:
new_df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
217191,140800.000,1.995,-0.399,-0.470,0.355,-0.560,-0.340,-0.599,0.085,1.474,-0.131,-1.101,-0.244,-1.222,0.214,0.847,0.297,-0.507,0.022,-0.036,-0.280,-0.204,-0.541,0.418,0.650,-0.449,-0.651,0.021,-0.027,8.000,0
199195,132850.000,-1.470,1.412,-0.415,-0.767,1.582,-0.836,1.783,-0.908,0.722,0.395,-0.086,0.251,0.939,-2.027,-0.037,-0.434,0.242,-0.291,-0.219,0.042,-0.393,-0.238,-0.058,0.582,0.471,-0.584,-1.186,-0.713,8.620,0
216676,140583.000,-0.917,0.884,0.102,-0.633,1.361,-1.066,1.497,-0.907,0.059,-0.097,0.125,-0.149,0.593,-1.630,0.438,0.044,0.177,0.106,-0.688,-0.221,0.195,1.041,0.002,1.077,-0.343,0.443,-0.815,-0.219,2.360,0
112379,72629.000,-0.808,1.411,1.191,-0.154,0.133,-0.079,-0.264,-2.846,-0.483,-0.591,-0.826,-0.001,-0.022,0.288,0.821,0.421,-0.546,-0.188,0.041,0.817,-1.567,-0.249,0.103,-0.151,-0.219,0.057,0.068,-0.071,1.980,0
66702,52176.000,-0.622,0.597,1.646,-0.415,0.244,-0.980,0.530,0.051,-0.601,-0.468,1.271,0.505,-0.608,0.470,-0.277,0.597,-0.810,0.083,-0.416,-0.108,-0.060,-0.376,0.088,0.499,-0.628,-0.147,0.076,0.145,0.890,0


In [16]:
new_df["Class"].value_counts()

Class
0    492
1    492
Name: count, dtype: int64

In [17]:
new_df.groupby("Class").mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,97430.476,0.228,-0.039,0.034,0.062,0.014,-0.046,0.085,0.010,-0.036,0.024,-0.045,0.045,-0.037,-0.049,-0.004,0.008,-0.026,-0.035,-0.043,0.020,-0.021,-0.042,0.008,-0.028,-0.015,-0.032,-0.020,-0.025,90.128
1,80746.807,-4.772,3.624,-7.033,4.542,-3.151,-1.398,-5.569,0.571,-2.581,-5.677,3.800,-6.259,-0.109,-6.972,-0.093,-4.140,-6.666,-2.246,0.681,0.372,0.714,0.014,-0.040,-0.105,0.041,0.052,0.171,0.076,122.211


In [18]:
X = new_df.drop(columns = "Class", axis = 1)
y = new_df["Class"]

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y, random_state = 2)

In [25]:
model = LogisticRegression()

In [26]:
model.fit(X_train, y_train)

LogisticRegression()

In [28]:
X_train_prediction = model.predict(X_train)
trainin_accuracy = accuracy_score(X_train_prediction, y_train)

In [29]:
trainin_accuracy

0.9440914866581956

In [30]:
X_test_prediction = model.predict(X_test)
test_accuracy = accuracy_score(X_test_prediction, y_test)

In [31]:
test_accuracy

0.9086294416243654